In [30]:
!pip install pytorch_lightning
!pip install nlp
!pip install transformers
!pip install flax

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation
import textwrap
from tqdm.auto import tqdm
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from nlp import load_metric

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)

/home/user01/miniconda3/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[nltk_data] Downloading package punkt to /home/user01/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_path = 'train.csv'
test_path  = 'validation.csv'


In [3]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [4]:

def concatenate(row):
    return 'متن: ' + row['context'] + '، پرسش: ' + row['question']

def format_df(k):
    # Apply the function to create a new 'input' column
    k['input'] = k.apply(concatenate, axis=1)

    # Extract the first answer from the 'answers' column
    k['output'] = k['answers']

    # Create a new DataFrame with 'input' and 'output' columns
    df = pd.DataFrame({'input': k['input'], 'output': k['output']})
    return df


In [5]:
train_data = format_df(train_df)
test_data = format_df(test_df)

In [7]:
test_data[input]

                                                  input  \
0     متن: نورمن ها (نورمن: Nourmands ؛ فرانسوی: Nor...   
1     متن: نورمن ها (نورمن: Nourmands ؛ فرانسوی: Nor...   
2     متن: نورمن ها (نورمن: Nourmands ؛ فرانسوی: Nor...   
3     متن: سلسله نورمن تأثیر عمده سیاسی ، فرهنگی و ن...   
4     متن: سلسله نورمن تأثیر عمده سیاسی ، فرهنگی و ن...   
...                                                 ...   
3230  متن: هندوانه یا تربوز (نام علمی: Citrullus lan...   
3231  متن: هندوانه یا تربوز (نام علمی: Citrullus lan...   
3232  متن: هندوانه یا تربوز (نام علمی: Citrullus lan...   
3233  متن: هندوانه یا تربوز (نام علمی: Citrullus lan...   
3234  متن: هندوانه یا تربوز (نام علمی: Citrullus lan...   

                                                 output  
0                                                فرانسه  
1                                          قرون 10 و 11  
2                               دانمارک ، ایسلند و نروژ  
3                                           ویلیام فاتح  
4

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/user01/miniconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:3361 in get_loc  │
│                                                                                                  │
│   3358 │   │   │   │   )                                                                         │
│   3359 │   │   │   casted_key = self._maybe_cast_indexer(key)                                    │
│   3360 │   │   │   try:                                                                          │
│ ❱ 3361 │   │   │   │   return self._engine.get_loc(casted_key)                                   │
│   3362 │   │   │   except KeyError as err:                                                       │
│   3363 │   │   │   │   raise KeyError(key) from err                                              │
│   3364                                                                                           │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:76                                                     │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:108                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5198                                        │
│                                                                                                  │
│ in pandas._libs.hashtable.PyObjectHashTable.get_item:5206                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: '0'

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 test_data[input]                                                                             │
│   2                                                                                              │
│                                                                                                  │
│ /home/user01/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:3458 in __getitem__     │
│                                                                                                  │
│    3455 │   │   if is_single_key:                                                                │
│    3456 │   │   │   if self.columns.nlevels > 1:                                                 │
│    3457 │   │   │   │   return self._getitem_multilevel(key)                                     │
│ ❱  3458 │   │   │   indexer = self.columns.get_loc(key)                                          │
│    3459 │   │   │   if is_integer(indexer):                                                      │
│    3460 │   │   │   │   indexer = [indexer]                                                      │
│    3461 │   │   else:                                                                            │
│                                                                                                  │
│ /home/user01/miniconda3/lib/python3.7/site-packages/pandas/core/indexes/base.py:3363 in get_loc  │
│                                                                                                  │
│   3360 │   │   │   try:                                                                          │
│   3361 │   │   │   │   return self._engine.get_loc(casted_key)             

In [ ]:
df = df.iloc[50000:,:]

In [6]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(42)

In [7]:
class bigbang(Dataset):
    def __init__(self, df, tokenizer, input_length, output_length):         
        self.dataset =  df
        self.input_length = input_length
        self.tokenizer = tokenizer
        self.output_length = output_length
    def __len__(self):
        return self.dataset.shape[0]
    def clean_text(self, text):
        text = text.replace('\n','')
        return text
    def convert_to_features(self, example_batch):
        input_ = self.clean_text(example_batch['input'])
        target_ = self.clean_text(example_batch['output'])
        source = self.tokenizer.batch_encode_plus([input_], max_length=self.input_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")
        targets = self.tokenizer.batch_encode_plus([target_], max_length=self.output_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")
        return source, targets
    def __getitem__(self, index):
        source, targets = self.convert_to_features(self.dataset.iloc[index])
        source_ids = source["input_ids"].squeeze()
        target_ids = targets["input_ids"].squeeze()
        src_mask    = source["attention_mask"].squeeze()
        target_mask = targets["attention_mask"].squeeze()
        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}  

In [10]:
model_path = 'parsT5'
configs = {
  "local_files_only": True,
  "from_flax": True
}

In [13]:
!pip  install lfs
!git clone https://huggingface.co/Ahmad/parsT5

fatal: destination path 'parsT5' already exists and is not an empty directory.


In [ ]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_path, **configs)#('Ahmad/parsT5-base')#('google/mt5-base')
dataset = bigbang(train_data, tokenizer, 512, 150)
len(dataset)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tokenizer = AutoTokenizer.from_pretrained(model_path, **configs)#('Ahmad/parsT5-base')#(     │
│   2 dataset = bigbang(train_data, tokenizer, 512, 150)                                           │
│   3 len(dataset)                                                                                 │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model_path' is not defined

In [12]:
data = dataset[50]
print()
print("Shape of Tokenized Text: ", data['source_ids'].shape)
print()
print("Sanity check - Decode Text: ", tokenizer.decode(data['source_ids']))
print("====================================")
print("Sanity check - Decode Summary: ", tokenizer.decode(data['target_ids']))


Shape of Tokenized Text:  torch.Size([512])

Sanity check - Decode Text:  متن: در 4 آوریل 2008 ، بیانسه با جی ز ازدواج کرد. او ازدواج خود را در مونتاژ ویدیویی در مهمانی گوش دادن به سومین آلبوم استودیویی خود ، من هستم... ساشا فیرس ، در 22 دسامبر 2008 در باشگاه سونی منهتن آشکار کرد. من هستم... ساشا فیرس در 18 نوامبر 2008 در ایالات متحده آزاد شد. این آلبوم رسماً آلترایگو بیانسه را معرفی می کند ، ساشا فییرس ، که هنگام ساخت تک آهنگ "دیوانه عاشق" در سال 2003 تصور شد ، در هفته اول 482،000 نسخه فروش ، اولین نمایش در بالای بیلبورد 200 ، و سومین آلبوم شماره یک پی در پی به بیانسه در آمریکا این آلبوم شامل آهنگ شماره یک "single ladies (put a ring on it)" و پنج آهنگ برتر "if i are a boy" و "halo" بود. موفقیت "halo" در ایالات متحده با دستیابی به موفقیت در تبدیل شدن به طولانی ترین تک آهنگ داغ 100 خود در کارنامه خود ، به بیانسه کمک کرد تا در طول دهه 2000 بیش از هر زن دیگری در لیست 10 تک آهنگ برتر این لیست قرار گیرد. این فیلم همچنین شامل "رویاهای شیرین" و تک آهنگهای "diva" ، "ego" ، "girl broken-hearte

In [40]:
def get_dataset(df, tokenizer, args):
      return bigbang(df, tokenizer=tokenizer, input_length=args.max_input_length, 
                        output_length=args.max_output_length)

In [14]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.hparamss = hparams        
        self.tokenizer = AutoTokenizer.from_pretrained("model2")
        self.model = T5ForConditionalGeneration.from_pretrained("model2")
        # self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        # self.tokenizer = AutoTokenizer.from_pretrained(hparams.tokenizer_name_or_path)
#         self.rouge_metric = load_metric('rouge')  
        n_observations_per_split = {
            "train": self.hparamss.n_train,
            "validation": self.hparamss.n_val,
            "test": self.hparamss.n_test,
        }
        self.n_obs = {k: v if v >= 0 else None for k, v in n_observations_per_split.items()}
    def freeze_params(self, model):
        for par in model.parameters():
            par.requires_grad = False
    def lmap(self, f, x):
        """list(map(f, x))"""
        return list(map(f, x))
    def is_logger(self):
        return self.trainer.global_rank <= 0
    def parse_score(self, result):
        return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()} 
    def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
    )
    def _step(self, batch):
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100
        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels,
            decoder_attention_mask=batch['target_mask'])
        loss = outputs[0]
        return loss
    def ids_to_clean_text(self, generated_ids):
        gen_text = self.tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        return self.lmap(str.strip, gen_text)
    def _generative_step(self, batch) :
        t0 = time.time()
        generated_ids = self.model.generate(
            batch["source_ids"],
            attention_mask=batch["source_mask"],
            use_cache=True,
            decoder_attention_mask=batch['target_mask'],
            max_length=150, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True)
        preds = self.ids_to_clean_text(generated_ids)
        target = self.ids_to_clean_text(batch["target_ids"]) 
        gen_time = (time.time() - t0) / batch["source_ids"].shape[0]  
        loss = self._step(batch)
        base_metrics = {'val_loss': loss}
        summ_len = np.mean(self.lmap(len, generated_ids))
        base_metrics.update(gen_time=gen_time, gen_len=summ_len, preds=preds, target=target)
#         self.rouge_metric.add_batch(preds, target)
        return base_metrics
    def training_step(self, batch, batch_idx):
        loss = self._step(batch)
        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}
    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
    def validation_step(self, batch, batch_idx):
        return self._generative_step(batch)
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
#         rouge_results = self.rouge_metric.compute() 
#         rouge_dict = self.parse_score(rouge_results)
#         tensorboard_logs.update(rouge1=rouge_dict['rouge1'], rougeL=rouge_dict['rougeL'])
        self.target_gen= []
        self.prediction_gen=[]
        return {"avg_val_loss": avg_loss, 
#                 "rouge1" : rouge_results['rouge1'],
#                 "rougeL" : rouge_results['rougeL'],
                "log": tensorboard_logs, 'progress_bar': tensorboard_logs}
    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparamss.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparamss.learning_rate, eps=self.hparamss.adam_epsilon)
        self.opt = optimizer
        return [optimizer]
    def optimizer_step(self,
                     epoch=None,
                     batch_idx=None,
                     optimizer=None,
                     optimizer_idx=None,
                     optimizer_closure=None,
                     on_tpu=None,
                     using_native_amp=None,
                     using_lbfgs=None):
        optimizer.step()
        optimizer.zero_grad()
        optimizer_closure()
        self.lr_scheduler.step()
    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}
        return tqdm_dict
    def train_dataloader(self):   
        n_samples = self.n_obs['train']
        train_dataset = get_dataset(train_data, tokenizer=self.tokenizer, args=self.hparamss)
        dataloader = DataLoader(train_dataset, batch_size=self.hparamss.train_batch_size, drop_last=True, shuffle=True, num_workers=2)
        t_total = (
            (len(dataloader.dataset) // (self.hparamss.train_batch_size * max(1, self.hparamss.n_gpu)))
            // self.hparamss.gradient_accumulation_steps
            * float(self.hparamss.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparamss.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader
    def val_dataloader(self):
        n_samples = self.n_obs['validation']
        validation_dataset = get_dataset(test_data,tokenizer=self.tokenizer, args=self.hparamss)
        return DataLoader(validation_dataset, batch_size=self.hparamss.eval_batch_size, num_workers=2)
    def test_dataloader(self):
        n_samples = self.n_obs['test']
        test_dataset = get_dataset(test_data, tokenizer=self.tokenizer, args=self.hparamss)
        return DataLoader(test_dataset, batch_size=self.hparamss.eval_batch_size, num_workers=2)

In [15]:
logger = logging.getLogger(__name__)
class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))
# logger = LoggingCallback()

In [16]:
from pytorch_lightning.loggers import TensorBoardLogger

logger = TensorBoardLogger("logs/", name="my_model")

In [17]:
args_dict = dict(
    output_dir="model", 
    model_name_or_path='Ahmad/parsT5',
    tokenizer_name_or_path='Ahmad/parsT5',
    max_input_length=512,
    max_output_length=150,
    freeze_encoder=False,
    freeze_embeds=False,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=1,
    gradient_accumulation_steps=8,
    n_gpu=1,
    resume_from_checkpoint=None, 
    val_check_interval = 10, 
    limit_val_batches = 0,
    n_val=5,
    n_train=-1,
    n_test=-1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

In [23]:
!mkdir -p t5_bigbang

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
args_dict.update({ 'num_train_epochs':4,
                 'train_batch_size': 8, 'eval_batch_size': 8})
args = argparse.Namespace(**args_dict)
print(args_dict)

{'output_dir': 'model', 'model_name_or_path': 'Ahmad/parsT5', 'tokenizer_name_or_path': 'Ahmad/parsT5', 'max_input_length': 512, 'max_output_length': 150, 'freeze_encoder': False, 'freeze_embeds': False, 'learning_rate': 0.0003, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 8, 'eval_batch_size': 8, 'num_train_epochs': 4, 'gradient_accumulation_steps': 8, 'n_gpu': 1, 'resume_from_checkpoint': None, 'val_check_interval': 10, 'limit_val_batches': 0, 'n_val': 5, 'n_train': -1, 'n_test': -1, 'early_stop_callback': False, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42}


In [19]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args.output_dir, monitor="val_loss", mode="min", save_top_k=3
)
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=5,
    # early_stop_callback=False,
    precision= 32,
    amp_backend='apex',
    amp_level=args.opt_level,
    resume_from_checkpoint=args.resume_from_checkpoint,
    gradient_clip_val=args.max_grad_norm,
    val_check_interval=args.val_check_interval,
    num_sanity_val_steps=0 ,
    limit_val_batches = 0,
    logger=logger,
    callbacks=[LoggingCallback()],
)

In [20]:
model = T5FineTuner(args)

In [59]:
model.model.save_pretrained('parsT5')


In [21]:
trainer = pl.Trainer(**train_params)

/home/user01/miniconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:407: LightningDeprecationWarning: The NVIDIA/apex AMP implementation has been deprecated upstream. Consequently, its integration inside PyTorch Lightning has been deprecated in v1.9.0 and will be removed in v2.0.0. The `Trainer(amp_backend='apex')` argument is deprecated. Removing this argument will avoid this message, it will select PyTorch's implementation automatically.
  "The NVIDIA/apex AMP implementation has been deprecated upstream. Consequently, its integration inside"
/home/user01/miniconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:418: LightningDeprecationWarning: The NVIDIA/apex AMP implementation has been deprecated upstream. Consequently, its integration inside PyTorch Lightning has been deprecated in v1.9.0 and will be removed in v2.0.0. The `Trainer(amp_level='O1')` argument is deprecated. Removing this argumen

In [22]:
trainer.fit(model)

/home/user01/miniconda3/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:93: UserWarning: When using `Trainer(accumulate_grad_batches != 1)` and overriding `LightningModule.optimizer_{step,zero_grad}`, the hooks will not be called on every batch (rather, they are called on every optimization step).
  "When using `Trainer(accumulate_grad_batches != 1)` and overriding"
You are using a CUDA device ('GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/user01/miniconda3/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorc

Epoch 0:   0%|                                         | 0/6196 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Epoch 0:   0%|          | 7/6196 [00:04<1:00:17,  1.71it/s, loss=0.831, v_num=1]

/home/user01/miniconda3/lib/python3.7/site-packages/pytorch_lightning/loops/optimization/optimizer_loop.py:363: LightningDeprecationWarning: The NVIDIA/apex AMP implementation has been deprecated upstream. Consequently, its integration inside PyTorch Lightning has been deprecated in v1.9.0 and will be removed in v2.0.0. The `T5FineTuner.optimizer_step()` hook is overridden, including the `using_native_amp` argument. Removing this argument will avoid this message, you can expect it to return True.
  "The NVIDIA/apex AMP implementation has been deprecated upstream. Consequently, its integration inside"


Epoch 1:   0%|                     | 0/6196 [00:00<?, ?it/s, loss=1.01, v_num=1]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environ

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|█████████| 6196/6196 [39:32<00:00,  2.61it/s, loss=0.767, v_num=1]
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
!mkdir model_final
model.model.save_pretrained('./model_final')
model.tokenizer.save_pretrained('./model_final')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


('./model_final/tokenizer_config.json',
 './model_final/special_tokens_map.json',
 './model_final/tokenizer.json')

In [99]:
# model2.model.from_pretrained('./model2')
model2.tokenizer.from_pretrained('./model2')

T5TokenizerFast(name_or_path='./model2', vocab_size=32003, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>'

### testing

In [ ]:
df = df_test

In [9]:
tokenizer = AutoTokenizer.from_pretrained('./model_final')
model = T5ForConditionalGeneration.from_pretrained('./model_final')
dataset = bigbang(test_data, tokenizer, 512, 150)

In [98]:
tokenizer.save_pretrained("./model2")

('./model2/tokenizer_config.json',
 './model2/special_tokens_map.json',
 './model2/tokenizer.json')

In [25]:
model.to('cuda')
dec, targets = [], []
for i in range(len(test_data)//32):
    loader = DataLoader(dataset, 32, shuffle=True)
    it = iter(loader)
    batch = next(it)
    batch["source_ids"].shape
    outs = model.generate(
                batch["source_ids"].cuda(),
                attention_mask=batch["source_mask"].cuda(),
                use_cache=True,
                decoder_attention_mask=batch['target_mask'].cuda(),
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
            )

    dec = dec + [tokenizer.decode(ids) for ids in outs]

# texts = [tokenizer.decode(ids) for ids in batch['source_ids']]
    targets = targets + [tokenizer.decode(ids) for ids in batch['target_ids']]

In [26]:
d_test = pd.DataFrame()
d_test['pred'] = dec
d_test['target'] = targets

In [27]:
d_test['pred'] = [i.replace('<pad>', '').replace('</s>', '').strip() for i in d_test['pred']]
d_test['target'] = [i.replace('<pad>', '').replace('</s>', '') for i in d_test['target']]

In [28]:
d_test 

,pred,target
0,socal,socal
1,ویرجینیا,شرکت اوهایو ویرجینیا
2,22 نوامبر,22 نوامبر
3,نورمن ها و نورمن,نورمن
4,دلتا,راین قدیمی
...,...,...
3227,منطقه ای در جنوب شرق اروپا,منطقه ای در جنوب شرق اروپا
3228,باکتریهای بی هوازی,باکتریهای بی هوازی
3229,دهه 1960,دهه 1960
3230,ورشو,ورشو


In [29]:
d_test.to_csv('./model/outs.csv')

In [10]:
d_test = pd.read_csv('./model/outs.csv')

In [19]:
from nltk.translate.bleu_score import sentence_bleu


def compute_f1(prediction, answer):
    pred_tokens = prediction.split()
    answer_tokens = answer.split()
    
    if len(pred_tokens) == 0 or len(answer_tokens) == 0:
        return int(pred_tokens == answer_tokens)
    
    common_tokens = set(pred_tokens) & set(answer_tokens)
    
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(answer_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(answer_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

def compute_exact_match(prediction, answer):
    return int(prediction == answer)

def bleu(prediction, answer) : 
  reference = [answer.split(' ')]
  candidate = pred.split(' ')
  BLEU = sentence_bleu(reference, candidate)
  BLEU1 = sentence_bleu(reference, candidate, weights=(1, 0, 0, 0))
  BLEU4 = sentence_bleu(reference, candidate, weights=(0, 0, 0, 1))

  return BLEU, BLEU1, BLEU4


In [20]:
EH = 0
F1 = 0
BLEU = 0
BLEU1 = 0
BLEU4 = 0
for idx, row in d_test.iterrows():
    target = row['target']
    pred = row['pred']
    if target == '':
        if pred == 'بدون پاسخ':
            EH += 1
            F1 += 1
            BLEU += 1
            BLEU1 += 1
            BLEU4 += 1
        continue

    EH += compute_exact_match(pred, target)
    F1 += compute_f1(pred, target)
    b, b1, b4 = bleu(pred, target)
    BLEU += b
    BLEU1 += b1
    BLEU4 += b4

EH /= len(d_test)
F1 /= len(d_test)
BLEU /= len(d_test)
BLEU1 /= len(d_test)
BLEU4 /= len(d_test)
 

In [22]:
print('Exact match of ParsT5 on testset : ' + str(EH))
print('F1 score of ParsT5 on testset : ' + str(F1))
# print('BLEU score of ParsT5 on testset : ' + str(BLEU))
print('BLEU1 score of ParsT5 on testset : ' + str(BLEU1))
# print('BLEU4 score of ParsT5 on testset : ' + str(BLEU4))
     


Exact match of ParsT5 on testset : 0.41646039603960394
F1 score of ParsT5 on testset : 0.5844615579920778
BLEU1 score of ParsT5 on testset : 0.5331487150009248
